In [1]:
# Import Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
import tarfile
import re
import copy
import pickle
import argparse
import os.path
import sys
import time
import gzip
import configparser
import csv
import numpy as np

In [3]:
def load_data_mnist(dataset):
    f = gzip.open(dataset, 'rb')
    train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    f.close()

    n_in = 28
    in_channel = 1

    test_set_x, test_set_y = test_set
    valid_set_x, valid_set_y = valid_set
    train_set_x, train_set_y = train_set

    rval = [(train_set_x, train_set_y), (valid_set_x, valid_set_y), (test_set_x, test_set_y)]
    return rval, n_in, in_channel, 10


def load_data_cifar10(dataset):
    train_dic = dict()
    f = tarfile.open(dataset, 'r')
    for member in f.getmembers():
        match1 = re.search(r".*\/data_batch_.*", member.name)
        match2 = re.search(r".*\/test_batch", member.name)
        if match1 is not None:
            print("Training: extracting {} ...".format(member.name))
            ef = f.extractfile(member)
            train_tmp = pickle.load(ef, encoding='latin1')
            if bool(train_dic) is False:
                train_dic = train_tmp
            else:
                train_dic['data'] = np.append(train_dic['data'], train_tmp['data'], axis=0)
                train_dic['labels'] = np.append(train_dic['labels'], train_tmp['labels'], axis=0)
        elif match2 is not None:
            print("Testing/Validating: extracting {} ...".format(member.name))
            ef = f.extractfile(member)
            test_dic = pickle.load(ef, encoding='latin1')
            test_dic['labels'] = np.array(test_dic['labels'])
    f.close()

    n_in = 32
    in_channel = 3

    def shared_dataset(data_dic, borrow=True):
        data_x = data_dic['data'].astype(np.float32)
        data_x /= 255.0#data_x.max()
        data_y = data_dic['labels'].astype(np.int32)
        return data_x, data_y

    train_set_x, train_set_y = shared_dataset(train_dic)
    test_set_x, test_set_y = shared_dataset(test_dic)
    #valid_set_x, valid_set_y = shared_dataset(valid_dic)

    rval = [(train_set_x, train_set_y), (test_set_x, test_set_y), (test_set_x, test_set_y)]
    return rval, n_in, in_channel, 10



In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as N
import matplotlib.pyplot as plt
import os
import numpy as np
input_path = os.path.join('/content/drive/MyDrive','MNIST_data/mnist.pkl.gz')
drive_path = os.path.join('/content/drive/myDrive')

Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
OUTPUT_FOLDER = "output"
EXERCISE = "exercise_2"
OUTPUT_DIR = os.path.join(drive_path, OUTPUT_FOLDER)
# os.makedirs(OUTPUT_DIR, exist_ok=True)

In [6]:
# Read data from file
data_input = load_data_mnist(input_path)
data = data_input[0]

# Data layout changes since output should an array of 10 with probabilities
real_output = N.zeros( (N.shape(data[0][1])[0] , 10), dtype=N.float32 )
for i in range ( N.shape(data[0][1])[0] ):
  real_output[i][data[0][1][i]] = 1.0

# Data layout changes since output should an array of 10 with probabilities
real_check = N.zeros( (N.shape(data[2][1])[0] , 10), dtype=N.float32 )
for i in range ( N.shape(data[2][1])[0] ):
  real_check[i][data[2][1][i]] = 1.0

# Set up the computation. Definition of the variables.
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(x, W) + b)
y_ = tf.placeholder(tf.float32, [None, 10])

cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))


In [10]:
experiments = {"Gradient_Descent": {"lr": [0.01, 0.1, 0.5, 1.0]},
               "Adam": {"lr": [0.000001, 0.0001, 0.001, 0.005]},
              "Momentum-0.2": {"lr": [0.001, 0.01, 0.05, 0.1],
                           "momentum": 0.2},
              "Momentum-0.4": {"lr": [0.001, 0.01, 0.05, 0.1],
                           "momentum": 0.4},
              "Momentum-0.6": {"lr": [0.001, 0.01, 0.05, 0.1],
                           "momentum": 0.6},
              "Momentum-0.8": {"lr": [0.001, 0.01, 0.05, 0.1],
                           "momentum": 0.8}}
N_EPOCHS = 30

In [14]:
for exercise_name, parameters in experiments.items():

  for LEARNING_RATE in parameters["lr"]:

    with tf.device('/gpu:0'):
      # optimizer
      if "Gradient_Descent" in exercise_name:
        train_step = tf.train.GradientDescentOptimizer(LEARNING_RATE).minimize(cross_entropy)
      elif "Adam" in exercise_name:
        train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
      elif "Momentum" in exercise_name:
        train_step = tf.train.MomentumOptimizer(LEARNING_RATE, momentum=parameters["momentum"]).minimize(cross_entropy)

      # train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)

      sess = tf.InteractiveSession()
      tf.global_variables_initializer().run()

      # TRAINING PHASE
      print("TRAINING")

      ls_loss = []
      ls_acc = []
      for epoch in range(N_EPOCHS):
        for i in range(500):
          batch_xs = data[0][0][100*i:100*i+100]
          batch_ys = real_output[100*i:100*i+100]
          sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

        # CALCULATING THE LOSS
        loss = sess.run(cross_entropy, feed_dict={x: data[0][0], y_: real_output})

        #CHECKING THE ERROR
        # print("ERROR CHECK")

        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        ACC = sess.run(accuracy, feed_dict={x: data[2][0], y_: real_check})
        print(f"EPOCH {epoch}({N_EPOCHS}) -- LR: {LEARNING_RATE} -- Loss: {loss} --- ACC: {ACC}")

        ls_loss.append(loss)
      ls_acc.append(ACC)
      sess.close()

    plt.plot(range(N_EPOCHS), ls_loss, label=f"lr = {LEARNING_RATE}")
    # Use log scale to better visualize the loss
    plt.yscale("log")

  title = exercise_name + " with different LR "
  if "momentum" in parameters:
    title += " (Mom - " + str(parameters["momentum"]) + ")"

  plt.legend()
  plt.title(title)
  plt.xlabel("Epochs")
  plt.ylabel("Loss")
  filename = exercise_name + "_lr_" + str(np.min(parameters["lr"])) + "_" + \
    str(np.max(parameters["lr"]))

  if "momentum" in parameters:
    filename += "_mom_" + str(parameters["momentum"])

  filename += ".png"
  plt.savefig(filename, bbox_inches="tight")
  plt.clf()


/usr/local/lib/python3.10/dist-packages/tensorflow/python/client/session.py:1793: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


TRAINING
EPOCH 0(30) -- LR: 0.01 -- Loss: 0.8099751472473145 --- ACC: 0.8504999876022339
EPOCH 1(30) -- LR: 0.01 -- Loss: 0.6193473935127258 --- ACC: 0.8718000054359436
EPOCH 2(30) -- LR: 0.01 -- Loss: 0.5414425134658813 --- ACC: 0.8808000087738037
EPOCH 3(30) -- LR: 0.01 -- Loss: 0.4972866475582123 --- ACC: 0.8859000205993652
EPOCH 4(30) -- LR: 0.01 -- Loss: 0.4681302607059479 --- ACC: 0.8906999826431274
EPOCH 5(30) -- LR: 0.01 -- Loss: 0.447089284658432 --- ACC: 0.8935999870300293
EPOCH 6(30) -- LR: 0.01 -- Loss: 0.43099743127822876 --- ACC: 0.896399974822998
EPOCH 7(30) -- LR: 0.01 -- Loss: 0.4181773066520691 --- ACC: 0.8981000185012817
EPOCH 8(30) -- LR: 0.01 -- Loss: 0.4076499938964844 --- ACC: 0.9002000093460083
EPOCH 9(30) -- LR: 0.01 -- Loss: 0.3988015651702881 --- ACC: 0.9014000296592712
EPOCH 10(30) -- LR: 0.01 -- Loss: 0.3912256360054016 --- ACC: 0.9035999774932861
EPOCH 11(30) -- LR: 0.01 -- Loss: 0.3846413195133209 --- ACC: 0.9050999879837036
EPOCH 12(30) -- LR: 0.01 -- Lo

<Figure size 640x480 with 0 Axes>